In [375]:
import numpy as np
import pandas as pd

In [376]:
rotten = pd.read_json("rotten-tomatoes.json.gz", orient = 'record', lines = True)

In [377]:
genre = pd.read_json("genres.json.gz", orient = 'record', lines = True)

In [378]:
wiki_review = pd.read_json("wikidata-movies.json.gz", orient = 'record', lines = True)

In [379]:
data = pd.merge(wiki_review, rotten, on = 'imdb_id')

In [380]:
data = data[data['audience_ratings'] > 3000]

In [381]:
def replaceQ(n):
    new = n.replace("Q", "")
    return new
to_replace = np.vectorize(replaceQ)
genre['wikidata_id'] = genre['wikidata_id'].apply(to_replace)
genre['wikidata_id'] = genre['wikidata_id'].astype(int)


In [382]:
data = data.dropna(subset=['audience_average', 'audience_percent', 'critic_average', 'critic_percent'])


In [383]:
data['overall'] = (data['audience_average']/5.0)*30 + (data['audience_percent']/100)*10 + (data['critic_average']/10.0)*40 + (data['critic_percent']/100)*20


In [384]:
data.loc[data['overall'] >= 85, "success",]= 1
data.loc[data['overall'] < 85, "success",]= 0


In [385]:
genre_data = data['genre'].apply(pd.Series)
#genre_0 = genre.iloc[:,0]
#genre_0


In [386]:
genre_data = genre_data.fillna('0')


In [387]:
genre_data = genre_data.apply(to_replace)


In [388]:
genre['label'] = range(5559)
genre['label'] = genre['label']+1



In [389]:
X = np.array(genre_data)
y = np.array(data['success'])


In [390]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [391]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.9313008130081301

In [392]:
new_data = data
new_data = new_data.dropna(subset = ['cast_member'])


In [393]:
actor = new_data['cast_member'].apply(pd.Series)


In [394]:
actor = actor.fillna('0')
actor = actor.apply(to_replace)



In [395]:
M = np.array(actor)
n = np.array(new_data['success'])
M_train, M_test, n_train, n_test = train_test_split(M,n)

In [396]:
rf_actor = RandomForestClassifier()
rf_actor.fit(M_train,n_train)
rf_actor.score(M_test, n_test)

0.943259385665529